In [ ]:
from IPython.display import display, HTML

import xarray
from geocoreg import registrators, visualization as vz, xr_registration

In [ ]:
ma = xarray.open_zarr('./test_data.zarr')
ma = ma['test_data']
ma

In [ ]:
dst_img = ma.isel(time=0)

In [ ]:
ma = ma.load()

In [ ]:
# registrator = registrators.PCCRegistrator()
# registrator = registrators.KoniaRegistrator()
registrator='pcc'
ts_registrated = xr_registration.coregistrate_timeseries(ma, dst_img, registrator=registrator, registration_bands=['red',])

In [ ]:
vz.export_timeseries_animation(ma.sel(band=['red','green','blue']), 'time', filename='original_ts.gif', )
vz.export_timeseries_animation(ts_registrated.sel(band=['red','green','blue']), 'time', filename='registrated_ts.gif', )

In [ ]:
html_code = """
<div style="display: flex; justify-content: space-around; align-items: center;">
    <div style="text-align: center;">
        <h3>Before registration</h3>
        <img src="original_ts.gif" width="500">
    </div>
    <div style="text-align: center;">
        <h3>After registration</h3>
        <img src="registrated_ts.gif" width="500">
    </div>
</div>
"""

# Display the HTML
display(HTML(html_code))